# More advanced match-ups
**Author:** Eli Holmes (NOAA)</br>
Last updated: November 21, 2025

Now you have the basics of how to match up points, if you have a dataset that you have created with `ds = xr.open_dataset(chunks={})`. We use the `chunks{}` to ensure that dask is used and we don't grab more data chunks than we need to. 

Once we have `ds`, can do match ups for (Y, X) lat/lon points with for one point with
 `ds.sel(latitude=Y, logitude=X, method="nearest")` if ds is a regular grid data (level 3 data). If it is not regular grid data, so swath (level 2) data, then we need to use indices. So we have to add a step where we find the index of latitude and longitude that is closest to our point.  Once we have the (i,j) indices for our point, we can use `ds.isel(latitude=i, longitude=j)`. We can do this for either gridded or non-gridded ds.  
 
## What to do if we have lots of points?

If we have lots of points to match up, we could do something like this
```
for lat, lon in zip(lats, lons):
    ds.sel(latitude=lat, logitude=lon, method="nearest")
```
This going to be shockingly slow. Fine if you only have a few points, but terrible if you have 100s or 1000s. We would be loading the maximum amount of data. For each lat/lon pair, we are loading all the data to do the nearest calculation and we do this for every point!! The data are chunked in the cloud and we only want to load the chunks we need. Remember these files can be massive.  When working with data in the cloud, we want to minimize the I/O of files. Every chunk is a file.

**What do we do?**

1. We make sure we create `ds` with `chunks{}` so that dask knows the chunk structure. Check your `ds` with `ds.chunks` to see what the chunking is in the cloud bucket.
2. We compute the vector of indices for the points and use `ds.vindex([vector_i, vector_j)`. That will load only the needed chunks. If multiple points are in the same chunk, it will not load that chunk over and over. This will be much much faster for cloud data.


In [1]:
# Show example

## What if you need to open many files...

But what do with do if we have lots of points that are scattered across many (100s) of files in the cloud. We could try to open them all with `xr.open_mfdataset()` but that will be slow and crash our memory if we need to open 100s of files. We could try to open each file one by one in `for` loop, but opening a file, even if we do not load the data into memory, consumes memory that we cannot get back except by restarting the kernel. If you are opening 100s of files, you will either need a machine with massive RAM or you will have to work in chunks and restart the kernel periodically to get back memory. Both of those options are kind of awful and fortunately unnecessary. We can work in batches of files and use dask to start a cluster of workers and close the cluster after we are done with the batch. This will return our memory so we can happily chug through hundreds of files.

### Workflow

1. Create a function to process one file
2. Create a batch for loop that does
    * Grab a batch of file urls (to their location in the cloud)
    * Open a dask cluster
    * Process the files
    * Close the cluster when done
    * Grab the next batch of file urls
    * Repeat

You will probably want to save the extracted points after each batch in case the for loop crashes. Then you can go back to where you left off.

Almost working!

Use the dask distributed code to run process_one_file()

In [4]:
import pandas as pd
df = pd.read_parquet("na_surface_chla.parquet")
date_str = sorted(df["time"].dt.strftime("%Y-%m-%d").unique())

In [5]:
import earthaccess
auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)
import xarray as xr
results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    temporal = (df.time.min(), df.time.max()),
    granule_name="*.DAY.*.4km.nc"
)
fileset = earthaccess.open(results);

QUEUEING TASKS | :   0%|          | 0/560 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/560 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/560 [00:00<?, ?it/s]

In [7]:
import xarray as xr
ds = xr.open_dataset(
        fileset[0],
        chunks={},       # or your chunks
        cache=False,
    )
ds

<xarray.Dataset> Size: 26GB
Dimensions:     (wavelength: 172, lat: 4320, lon: 8640, rgb: 3,
                 eightbitcolor: 256)
Coordinates:
  * wavelength  (wavelength) float64 1kB 346.0 348.0 351.0 ... 714.0 717.0 719.0
  * lat         (lat) float32 17kB 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon         (lon) float32 35kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    Rrs         (lat, lon, wavelength) float32 26GB dask.array<chunksize=(16, 1024, 8), meta=np.ndarray>
    palette     (rgb, eightbitcolor) uint8 768B dask.array<chunksize=(3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240305.L3m.DAY.RRS.V3_1.Rrs...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/RRS/3.1
    keywords:                          Earth Science > Oceans > Ocean Optics ...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         3861548
    data_minimum:                      -0.009998
    data_maximum:                      0.092371605

In [4]:
# watch this blow up the memory. restart the kernel after it runs
import os, psutil
proc = psutil.Process(os.getpid())
def print_mem(tag):
    rss = proc.memory_info().rss / 1e6
    print(f"{tag}: RSS={rss:.1f} MB")

for k, uri in enumerate(fileset[0:6], start=1):
    ds = xr.open_dataset(uri)
    print_mem(f"After {k} files:")

After 1 files:: RSS=480.8 MB
After 2 files:: RSS=536.3 MB
After 3 files:: RSS=585.7 MB
After 4 files:: RSS=640.6 MB
After 5 files:: RSS=692.8 MB
After 6 files:: RSS=745.2 MB


In [9]:
## THIS WORKS
from dask.distributed import Client, LocalCluster

def run_batch(batch_uris):
    cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=1,
        processes=True,
        memory_limit="1.3GB",
    )
    client = Client(cluster)

    futures = client.map(process_one_file, batch_uris)
    results = client.gather(futures)
    client.cancel(futures)

    client.close()
    cluster.close()

    return results

all_results = []
batch_size = 10   # tune this up/down as your memory allows

for i in range(0, len(fileset), batch_size):
    batch = fileset[i:i+batch_size]
    print(f"Batch {i//batch_size+1}: {len(batch)} files")
    all_results.extend(run_batch(batch))


Batch 1: 10 files


/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34827 instead
  warnings.warn(


<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240305.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240306.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240307.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240308.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240309.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240310.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240311.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240312.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240315.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240316.L3m.DAY.R

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37455 instead
  warnings.warn(


<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240317.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240318.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240319.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240320.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240321.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240322.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240323.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240324.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240325.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20240328.L3m.DAY.R

...

<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250901.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250902.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250903.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250904.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250905.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250906.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250907.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250908.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250909.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250910.L3m.DAY.R

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45449 instead
  warnings.warn(
2025-11-21 21:40:15,777 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/worker.py", line 1269, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3

<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250911.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250912.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250913.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250914.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250915.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250916.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250917.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250918.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250919.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250920.L3m.DAY.R

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39797 instead
  warnings.warn(


<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250921.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250922.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250923.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250924.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250925.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250926.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250927.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250928.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250929.L3m.DAY.RRS.V3_1.Rrs.4km.nc>
<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_OCI.20250930.L3m.DAY.R

# NEW PROCESS FILE CODE

Simpler verison of `match-bargo`

This function assumes:

* The files are for a regular grid, i.e. level 3 data
* The files do not have a time coordinate. This is how the PACE level 3 daily and 8-day files are currently structured.
* We are looking at a variable, like Rrs, that has a 3rd dimension of wavelength or similar. Usually called wavelength or wavelength_3d. So the dims are (lat, lon, 3rd dim)
* `time_coverage_start` and `time_coverage_end` exist as attributes in the files.
* `ds[ds_vec_name]` is a 1D coordinate aligned with the last dimension of ds_var_name.

Process Steps:

1.  Subset our dataframe with our in-situ samples only those rows within in the  time window covered by the cloud-hosted xarray dataset (`ds`) that our url points to. Let's call that `df_day`
2.  Get the lat/lon indices of `ds` that correspond to the lat/lon rows in `df_day`. These indices will look like `[0, 3, 6]` and `[0, 10, 15]` corresponding to index pairs, like `(3, 10)` corresponding to the lat/lon pair in a row of `df_day`.
3.  Do a fast point retrieval from `ds`. `ds` is cloud optimized (chunked) file and we only want to the chunks that we need. We do not need to load all the chunks. I/O of cloud-hosted data is the slow part so don't touch more of the data than you need to.
4. Build a dataframe with the predictor variables that we want from `ds`.

In [34]:
# New process code
def process_one_match(
    uri, df,
    ds_lat_name="lat", ds_lon_name="lon", ds_time_name="time", 
    ds_vec_name="wavelength", ds_var_name="Rrs",
    df_lat_name="LATITUDE", df_lon_name="LONGITUDE", 
    df_time_name="TIME", df_y_name="log10_CHLA_surface"):
    
    import xarray as xr
    import pandas as pd

    with xr.open_dataset(uri, chunks={}, cache=False) as ds:

        # --- Step 1: subset df to the time window in ds
        t_start = pd.to_datetime(ds.attrs["time_coverage_start"], utc=True)
        t_end   = pd.to_datetime(ds.attrs["time_coverage_end"], utc=True)
        df_times = pd.to_datetime(df[df_time_name], utc=True)   
        df_day = df[(df_times >= t_start) & (df_times < t_end)]

        # Stop if no days in df match day in ds
        if df_day.empty: return None

        # --- Step 2: Get the indices of ds that correspond to the lat/lon rows in df_day
        # Get the index for lat/lon dims in ds
        lat_idx = ds.get_index(ds_lat_name)
        lon_idx = ds.get_index(ds_lon_name)
        # Get the lat/lon vals in df_day
        df_lats = df_day[df_lat_name].to_numpy(dtype=float)
        df_lons = df_day[df_lon_name].to_numpy(dtype=float)
        # Use pandas indexer functions to quickly find nearest indices
        lat_i = lat_idx.get_indexer(df_lats, method="nearest")
        lon_i = lon_idx.get_indexer(df_lons, method="nearest")

        # --- Step 3: Retrieve just the chunks of ds (in cloud) that we need
        # Fast point retreval from cloud optimized (chunked) data
        # this retrieves just data we want, so minimal I/O (I/O is slow)
        data_da = ds[ds_var_name].data  # (lat, lon, wavelength)
        var_vals = data_da.vindex[lat_i, lon_i, :].compute() # (n rows df_day, wavelengths)
    
        # --- Step 4: build a dataframe of what we need to train ---
        data = {
            ds_time_name: pd.to_datetime(df_day[df_time_name], utc=True),
            ds_lat_name:  ds[ds_lat_name].vindex[lat_i],
            ds_lon_name:  ds[ds_lon_name].vindex[lon_i],
            "y": df_day[df_y_name].to_numpy(),
            }

        # Get the wavelength values for our col names
        vec_vals = ds[ds_vec_name].values
        for j, v in enumerate(vec_vals):
            label = int(v) # make integer for nicer label
            col_name = f"{ds_var_name}_{label}"
            data[col_name] = var_vals[:, j].astype(float)

        return pd.DataFrame(data)
